In [1]:
import os
import json
from openai import OpenAI

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def get_current_weather(location, unit="섭씨"):
    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

messages = [{"role": "user", "content": "지금 서울날씨를 섭씨로 알려줘."}]

functions = [
    {
        "name": "get_current_weather",
        "description": "특정 지역의 날씨를 알려줍니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "지역이름. 예) 서울, 부산, 제주도",
                },
                "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
            },
            "required": ["location"],
        },
    }
]

In [21]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages,
    functions = functions,
    function_call = "auto",
)

response_message = response.choices[0].message

print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"서울","unit":"섭씨"}', name='get_current_weather'), tool_calls=None)


In [4]:
if response_message.function_call is not None:
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }
    function_name = response_message.function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(
        location=function_args['location'],
        unit=function_args['unit']
    )

    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )  # get a new response from GPT where it can see the function response

    print(second_response.choices[0].message.content)

현재 서울의 날씨는 섭씨 24도이며, 맑고 바람이 불고 있습니다.


In [5]:
messages = [{"role": "user", "content": "지금 대전 날씨를 섭씨로 알려줘."}]
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_current_weather",
            "description": "특정 지역의 날씨를 알려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "지역이름. 예) 서울, 부산, 제주도",
                    },
                    "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
                },
                "required": ["location"],
            },
        }
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto" if tools is not None else None
    )
response_message = response.choices[0].message
print(response_message)
print(response_message.tool_calls[0])
print(type(response_message.tool_calls[0]))
print(type(response_message.tool_calls[0].function))

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9u7VdIra004EVU2Ez1qe1i9L', function=Function(arguments='{"location":"대전","unit":"섭씨"}', name='get_current_weather'), type='function')])
ChatCompletionMessageToolCall(id='call_9u7VdIra004EVU2Ez1qe1i9L', function=Function(arguments='{"location":"대전","unit":"섭씨"}', name='get_current_weather'), type='function')
<class 'openai.types.chat.chat_completion_message_tool_call.ChatCompletionMessageToolCall'>
<class 'openai.types.chat.chat_completion_message_tool_call.Function'>


In [6]:
available_functions = {
    "get_current_weather": get_current_weather,
}

for tool_call in response_message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args['location'],
        unit=function_args['unit']
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "function",
            "name": function_name,
            "content": json.dumps(function_response, ensure_ascii=False)
        }
    )
    
second_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
print(second_response.choices[0].message.content)

현재 대전의 날씨는 약 24도 섭씨입니다. 맑고 바람이 부는 날씨입니다.


In [7]:
print(messages)

[{'role': 'user', 'content': '지금 대전 날씨를 섭씨로 알려줘.'}, {'tool_call_id': 'call_9u7VdIra004EVU2Ez1qe1i9L', 'role': 'function', 'name': 'get_current_weather', 'content': '"{\\"location\\": \\"\\\\ub300\\\\uc804\\", \\"temperature\\": \\"24\\", \\"unit\\": \\"\\\\uc12d\\\\uc528\\", \\"forecast\\": [\\"sunny\\", \\"windy\\"]}"'}]


In [23]:
!pip install gradio

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 38.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.1/128.1 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 KB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 63.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 KB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 KB 38.5 MB/s eta 0:00:00
    

In [27]:
import gradio as gr

def greet(name):
    return "ㅎㅇ " + name + "씨!"

demo = gr.Interface(
    fn = greet,
    inputs = gr.Textbox(),
    outputs = "text"
)


demo.launch(share = True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://e5ae8192f198687136.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]    
    ])

    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

demo = gr.Interface(sepia, gr.Image(width=200, height=200), "image")
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.10/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
  File "/home/ubuntu/.local/lib/python3.10/site-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/ubuntu/.local/lib/python3.10/site-packages/gradio/blocks.py", line 2235, in process_api
    result = await self.call_function(
  File "/home/ubuntu/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1746, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/home/ubuntu/.local/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/home/ubuntu/.local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 2470, in run_sync_in_worker_thread
    return await future
  File "/home/ubuntu/.lo

In [ ]:
import gradio as gr
import os
import time
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.update(value="", interactive=False)

def add_file(history, file):
    history = history + [((file.name,), None)]
    return history

def bot(history):
    user_message = history[-1][0]  # 마지막 사용자 입력

    # ✅ GPT에게 메시지 전달
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # 또는 "gpt-4" 등
        messages=[
            {"role": "system", "content": "당신은 친절한 AI 챗봇입니다."},
            {"role": "user", "content": user_message}
        ],
        stream=True  # ⏳ 스트리밍으로 출력
    )

    history[-1][1] = ""  # 답변 초기화

    for chunk in response:
        if 'choices' in chunk and len(chunk['choices']) > 0:
            delta = chunk['choices'][0]['delta']
            if 'content' in delta:
                history[-1][1] += delta['content']
                yield history  # 실시간 업데이트

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=(None, (os.path.join("avatar.png"))),
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="텍스트를 입력하고 엔터를 치거나 이미지를 업로드하세요",
            container=False,
        )
        btn = gr.UploadButton("Upload", file_types=["image", "video", "audio"])

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    txt_msg.then(lambda: gr.update(interactive=True), None, [txt], queue=False)
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.queue()
demo.launch(server_name='0.0.0.0')


/tmp/ipykernel_37877/1231079666.py:39: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipykernel_37877/1231079666.py:39: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


* Running on local URL:  http://0.0.0.0:7866
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.10/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
  File "/home/ubuntu/.local/lib/python3.10/site-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/ubuntu/.local/lib/python3.10/site-packages/gradio/blocks.py", line 2235, in process_api
    result = await self.call_function(
  File "/home/ubuntu/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1758, in call_function
    prediction = await utils.async_iteration(iterator)
  File "/home/ubuntu/.local/lib/python3.10/site-packages/gradio/utils.py", line 762, in async_iteration
    return await anext(iterator)
  File "/home/ubuntu/.local/lib/python3.10/site-packages/gradio/utils.py", line 753, in __anext__
    return await anyio.to_thread.run_sync(
  File "/home/ubuntu/.local/lib/python3.10/site-packages/anyio/to_thre